In [5]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product

import warnings
warnings.filterwarnings(action = 'ignore')

In [6]:
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [7]:
a = [1,2,3,4,5]
mean_a=sum(a)/5

def var_a(any_num,a):
  sum_list =[]
  for num in a:
    input_n = abs(num-any_num) # 절대 편차 적용
    sum_list.append(input_n)
  
  return sum(sum_list)/len(a)

# 모분산의 추정: 표본분산의 정의

앞의 모분포와 표본분포에서 분산과 평균의 경우를 살펴보자.
- 모분포: [-1, -1, 0, 0, 0, 0, 0, 1, 1, 1]

In [8]:
# 모분포 만들기
uni=[-1,-1,0,0,0,0,0,1,1,1]

avg = sum(uni)/len(uni)
middle = 0 
var = np.around(np.var(uni),2)
std = np.around(np.std(uni),2)
aad = var_a(np.median(uni),uni)

sample_list=[]
sample_list=list(product(set(uni),set(uni),set(uni)))

sample_df=pd.DataFrame(columns=['x1','x2','x3','확률','평균','중앙값','분산','표준편차','평균절대편차'])

for idx in range(len(sample_list)):
  x1 = sample_list[idx][0]
  x2 = sample_list[idx][1]
  x3 = sample_list[idx][2]

  x1_pro=uni.count(x1)/len(uni)
  x2_pro=uni.count(x2)/len(uni)
  x3_pro=uni.count(x3)/len(uni)

  # 관측치 대입
  sample_df.loc[idx,'x1'] = x1
  sample_df.loc[idx,'x2'] = x2
  sample_df.loc[idx,'x3'] = x3


  #확률 대입
  sample_df.loc[idx,'확률'] = np.around(x1_pro * x2_pro * x3_pro,3)

  #평균 대입
  sample_df.loc[idx,'평균'] = np.around(np.mean(sample_list[idx]),3)

  #중앙값 대입
  sample_df.loc[idx,'중앙값'] = np.median(sample_list[idx])

  #분산값 대입
  sample_df.loc[idx,'분산'] = np.var(sample_list[idx])

  #표준편차값 대입
  sample_df.loc[idx,'표준편차'] = np.std(sample_list[idx])

  #평균절대편차값 대입
  sample_df.loc[idx,'평균절대편차'] = var_a(np.median(sample_list[idx]),sample_list[idx])


sample_df = sample_df.sort_values(['x1','x2','x3']).reset_index(drop=True)

In [9]:
sample_df.head()

,x1,x2,x3,확률,평균,중앙값,분산,표준편차,평균절대편차
0,-1,-1,-1,0.008,-1,-1,0,0,0
1,-1,-1,0,0.02,-0.667,-1,0.222222,0.471405,0.333333
2,-1,-1,1,0.012,-0.333,-1,0.888889,0.942809,0.666667
3,-1,0,-1,0.02,-0.667,-1,0.222222,0.471405,0.333333
4,-1,0,0,0.05,-0.333,0,0.222222,0.471405,0.333333


동일한 분산을 나타내는 경우를 합쳐, 표본 분산의 분포를 구해보자.

In [10]:
vr_dev=sample_df.groupby(['분산'])['확률'].sum().reset_index()
vr_dev.T

,0,1,2,3
분산,0.00,0.222222,0.666667,0.888889
확률,0.16,0.570000,0.180000,0.090000


위의 표본분산의 분포에 대해 다시 평균, 중앙값, 분산, 평균절대편차를 구해보자.

In [11]:
vd_mean = sum(vr_dev['분산']*vr_dev['확률'])
vd_median = 0  # 누적확률분포로 생각해보면 0.5일 때 값은 0 이기 때문임
vd_var = np.around(sum(((vr_dev['분산'] - vd_mean)**2)*vr_dev['확률']),3) # ((관측치-평균)**2) * P
vd_std = np.around(vd_var**(1/2),3)
vd_mad = np.around(sum(abs(vr_dev['분산'] - vd_mean)*vr_dev['확률']),3)

print('표본분산의 평균', vd_mean)
print('표본분산의 중앙값',vd_median)
print('표본분산의 분산',vd_var)
print('표본분산의 표준편차',vd_std)
print('표본분산의 평균절대편차',vd_mad)
print('모분산:', np.var(uni))

표본분산의 평균 0.32666666666666666
표본분산의 중앙값 0
표본분산의 분산 0.073
표본분산의 표준편차 0.27
표본분산의 평균절대편차 0.224
모분산: 0.49000000000000005


표본분산 분포의 평균(기댓값)이 0.327로 모분산 0.49보다 작게 나오는 것을 볼 수 있다. 표본평균을 통해 모평균을 추정하기 마련인데, 그럴 경우에는 표본의 분산값의 기댓값이 모분산과 일치하지 않는다.

그 이유는 분산의 최대우도 추정치는 "평균"인 것을 알면 된다. 표본의 평균은 표본 평균이기 때문에 어떠한 다른 수가 들어와도 표본 평균을 넣은 분산의 값보다 크거나 같을 수 밖에 없다.

In [12]:
[-1,1,1]

print('모집단의 평균',sum(uni)/len(uni))
print('[-1,1,1]의 평균',sum([-1,1,1])/len([-1,1,1]))
print('[-1,1,1]의 의 원래 분산',var_a(0.3333333333333333,[-1,1,1]))
print('[-1,1,1]의 의 모평균을 적용한 분산',var_a(0.1,[-1,1,1]))

print()
'''각 표본마다의 평균이 있는데, 그걸 무시하고 모평균을 넣으면 당연히 값이 이상해지기 마련..!'''

모집단의 평균 0.1
[-1,1,1]의 평균 0.3333333333333333
[-1,1,1]의 의 원래 분산 0.888888888888889
[-1,1,1]의 의 모평균을 적용한 분산 0.9666666666666667



'각 표본마다의 평균이 있는데, 그걸 무시하고 모평균을 넣으면 당연히 값이 이상해지기 마련..!'

이를 해결할 수 있는 방법은 표본의 크기에서 1을 뺀 수로 나눠주는 것이다.

In [13]:
# 기존의 분산 공식의 분모인 3을 2로 바꿔 주자.
sample_df['분산'] = sample_df['분산']* (3/2) 

In [14]:
vr_dev=sample_df.groupby(['분산'])['확률'].sum().reset_index()
vr_dev.T

vd_mean = sum(vr_dev['분산']*vr_dev['확률'])
vd_median = 0  # 누적확률분포로 생각해보면 0.5일 때 값은 0 이기 때문임
vd_var = np.around(sum(((vr_dev['분산'] - vd_mean)**2)*vr_dev['확률']),3) # ((관측치-평균)**2) * P
vd_std = np.around(vd_var**(1/2),3)
vd_mad = np.around(sum(abs(vr_dev['분산'] - vd_mean)*vr_dev['확률']),3)

print('표본분산의 평균', vd_mean)
print('표본분산의 중앙값',vd_median)
print('표본분산의 분산',vd_var)
print('표본분산의 표준편차',vd_std)
print('표본분산의 평균절대편차',vd_mad)
print('모분산:', np.var(uni))

표본분산의 평균 0.49
표본분산의 중앙값 0
표본분산의 분산 0.163
표본분산의 표준편차 0.404
표본분산의 평균절대편차 0.335
모분산: 0.49000000000000005


분산에 대한 기댓값이 모분산과 일치한다.

모평균 대신 표본평균을 사용하여 모분산을 추정할 경우, 표본의 크기가 하나 줄어드는 효과가 있다고 생각하면 쉽다.